In [54]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [55]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [56]:
# # importing sweetviz
# import sweetviz as sv

# #analyzing the dataset
# advert_report = sv.analyze(test_df)
# #display the report
# advert_report.show_html('Advertising.html')

In [57]:
credit_features =[]
counter=1
for col in train_df.columns:
    print(f"{counter},{col}")
    counter=counter+1
    credit_features.append(col)

1,Unnamed: 0
2,index
3,loan_amnt
4,int_rate
5,installment
6,home_ownership
7,annual_inc
8,verification_status
9,loan_status
10,pymnt_plan
11,dti
12,delinq_2yrs
13,inq_last_6mths
14,open_acc
15,pub_rec
16,revol_bal
17,total_acc
18,initial_list_status
19,out_prncp
20,out_prncp_inv
21,total_pymnt
22,total_pymnt_inv
23,total_rec_prncp
24,total_rec_int
25,total_rec_late_fee
26,recoveries
27,collection_recovery_fee
28,last_pymnt_amnt
29,collections_12_mths_ex_med
30,policy_code
31,application_type
32,acc_now_delinq
33,tot_coll_amt
34,tot_cur_bal
35,open_acc_6m
36,open_act_il
37,open_il_12m
38,open_il_24m
39,mths_since_rcnt_il
40,total_bal_il
41,il_util
42,open_rv_12m
43,open_rv_24m
44,max_bal_bc
45,all_util
46,total_rev_hi_lim
47,inq_fi
48,total_cu_tl
49,inq_last_12m
50,acc_open_past_24mths
51,avg_cur_bal
52,bc_open_to_buy
53,bc_util
54,chargeoff_within_12_mths
55,delinq_amnt
56,mo_sin_old_il_acct
57,mo_sin_old_rev_tl_op
58,mo_sin_rcnt_rev_tl_op
59,mo_sin_rcnt_tl
60,mort_acc
61,mths_since_re

In [58]:
y_train_list = train_df["loan_status"]
y_train_list

0         low_risk
1         low_risk
2         low_risk
3         low_risk
4         low_risk
           ...    
12175    high_risk
12176    high_risk
12177    high_risk
12178    high_risk
12179    high_risk
Name: loan_status, Length: 12180, dtype: object

In [59]:
y_dummies_train= pd.get_dummies(y_train_list, drop_first = True)
print(y_dummies_train.columns)
y_dummies_train

Index(['low_risk'], dtype='object')


,low_risk
0,1
1,1
2,1
3,1
4,1
...,...
12175,0
12176,0
12177,0
12178,0


In [60]:
y_test_list = test_df["loan_status"]
y_test_list

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
          ...    
4697    high_risk
4698    high_risk
4699    high_risk
4700    high_risk
4701    high_risk
Name: loan_status, Length: 4702, dtype: object

In [61]:
y_dummies_test= pd.get_dummies(y_test_list, drop_first = True)
print(y_dummies_test.columns)
y_dummies_test

Index(['low_risk'], dtype='object')


,low_risk
0,1
1,1
2,1
3,1
4,1
...,...
4697,0
4698,0
4699,0
4700,0


In [62]:
# drop columns
train_df = train_df.drop(columns=["Unnamed: 0", "index", "pymnt_plan", "recoveries", "collection_recovery_fee", "policy_code", "num_tl_120dpd_2m", "tax_liens", "loan_status", "debt_settlement_flag"])
test_df = test_df.drop(columns=["Unnamed: 0", "index", "pymnt_plan", "recoveries", "collection_recovery_fee", "policy_code", "num_tl_120dpd_2m", "tax_liens", "loan_status"])

In [63]:
# Convert categorical data to numeric and separate target feature for training data
X_dummies_train= pd.get_dummies(train_df, drop_first = True)
print(X_dummies_train.columns)
X_dummies_train

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App,hardship_flag_Y
0,13375.0,0.1797,483.34,223000.0,29.99,0.0,0.0,15.0,0.0,39728.0,...,32000.0,170200.0,1,0,0,0,0,1,0,0
1,21000.0,0.1308,478.68,123000.0,11.26,2.0,0.0,16.0,0.0,9585.0,...,15900.0,35398.0,1,0,0,1,0,1,0,0
2,20000.0,0.1240,448.95,197000.0,11.28,0.0,0.0,12.0,0.0,16708.0,...,22600.0,90340.0,1,0,0,1,0,1,0,0
3,3000.0,0.1240,100.22,45000.0,18.08,0.0,0.0,12.0,1.0,8809.0,...,19900.0,15406.0,0,0,1,0,0,1,0,0
4,30000.0,0.1612,1056.49,133000.0,27.77,0.0,2.0,13.0,0.0,65420.0,...,79500.0,58778.0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,28000.0,28.42,0.0,0.0,15.0,0.0,9679.0,...,18700.0,19055.0,0,0,1,0,0,1,0,0
12176,15000.0,0.1774,540.34,50000.0,23.43,4.0,0.0,16.0,0.0,3193.0,...,13300.0,54824.0,0,0,1,0,1,1,0,0
12177,3600.0,0.1862,131.28,60000.0,28.80,0.0,1.0,14.0,0.0,11804.0,...,14800.0,53065.0,0,0,1,0,0,1,0,0
12178,15000.0,0.0881,475.68,62000.0,11.44,0.0,0.0,5.0,0.0,157.0,...,7000.0,32930.0,1,0,0,1,0,1,1,0


In [64]:
# Convert categorical data to numeric and separate target feature for testing data
X_dummies_test= pd.get_dummies(test_df, drop_first = True)
print(X_dummies_test.columns)
X_dummies_test

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,initial_list_status_w,application_type_Joint App,hardship_flag_Y
0,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,9471.0,...,74600.0,99475.0,1,0,0,0,0,1,0,0
1,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,1280.0,...,5900.0,23628.0,0,0,1,0,0,1,0,0
2,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,4757.0,...,7300.0,15000.0,0,0,1,0,0,1,0,0
3,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,12731.0,...,13800.0,35981.0,0,0,1,0,0,1,0,0
4,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,10413.0,...,21000.0,24977.0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,30000.0,0.1240,673.42,140480.0,15.74,0.0,0.0,20.0,0.0,23215.0,...,48400.0,107388.0,0,0,1,1,0,0,0,0
4698,24000.0,0.0756,747.22,50000.0,26.81,0.0,0.0,9.0,0.0,459.0,...,13300.0,30775.0,0,0,1,0,0,1,0,0
4699,10000.0,0.2305,387.36,33000.0,38.51,0.0,2.0,7.0,0.0,6342.0,...,8500.0,29550.0,0,0,1,0,1,0,0,0
4700,8000.0,0.1862,205.86,38000.0,16.36,0.0,1.0,8.0,1.0,11636.0,...,1500.0,9657.0,0,0,1,1,0,1,0,0


## Consider the models

I believe the train data will perform better

In [65]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression()
classifier.fit(X_dummies_train, y_dummies_train)
print(f"Training Data Score: {classifier.score(X_dummies_train, y_dummies_train)}")
print(f"Testing Data Score: {classifier.score(X_dummies_test, y_dummies_test)}")

C:\Users\16309\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Training Data Score: 0.6487684729064039
Testing Data Score: 0.5153126329221608


C:\Users\16309\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [66]:
y_true = y_dummies_test
y_pred = classifier.predict(X_dummies_test)
confusion_matrix(y_true, y_pred)

array([[ 701, 1650],
       [ 629, 1722]], dtype=int64)

In [67]:
# Train a Random Forest Classifier model and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_dummies_train, y_dummies_train)
print(f'Training Score: {clf.score(X_dummies_train, y_dummies_train)}')
print(f'Testing Score: {clf.score(X_dummies_test, y_dummies_test)}')

<ipython-input-67-a30c7cbb312c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_dummies_train, y_dummies_train)


Training Score: 1.0
Testing Score: 0.6403658017864738


In [68]:
# Scale the data
scaler = StandardScaler()
X_dummies_train_scaled = scaler.fit_transform(X_dummies_train)
X_dummies_test_scaled = scaler.fit_transform(X_dummies_test)

In [69]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier.fit(X_train_scaled, y_dummies_train)
print(f"Training Data Score: {classifier.score(X_dummies_train_scaled, y_dummies_train)}")
print(f"Testing Data Score: {classifier.score(X_dummies_test_scaled, y_dummies_test)}")

C:\Users\16309\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Training Data Score: 0.708128078817734
Testing Data Score: 0.6626967247979583


C:\Users\16309\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [70]:
# Train a Random Forest Classifier model on the scaled data and print the model score
print(f'Testing Score: {clf.score(X_dummies_test_scaled, y_dummies_test)}')

Testing Score: 0.5
